In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df_target=data.target
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df_target=data.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df,df_target, test_size=0.2, random_state=42)
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)
import math

In [75]:
class bee_colony:

  def __init__(self,objective_function,n_iteration=50,population_size=50):
    self.population_size = population_size
    self.n_iteration=n_iteration
    self.objective_function=objective_function

  def initialize_population(self,X):
    self.food_source=np.zeros((X_train.shape[1],X_train.shape[1]))
    for i in range(X_train.shape[1]):
      self.food_source[i][i]=1

  def evaluate_fitness(self,model,X_train,y_train,X_valid,y_valid):
        scores =  []
        for i,individual in enumerate(self.food_source):
            chosen_features = [index for index in range(X_train.shape[1]) if individual[index]==1]
            X_train_copy = X_train.iloc[:,chosen_features]
            X_valid_copy = X_valid.iloc[:,chosen_features]
            score = self.objective_function(model,X_train_copy,y_train,X_valid_copy,y_valid)
            if score< self.best_score:
              self.best_score=score
              self.best_score_dimension=individual
            scores.append(score)
        self.fitness_scores = np.array(scores)

  def return_fitness(self,model,X_train,y_train,X_valid,y_valid,individuals_array):
        scores =  []
        for i,individual in enumerate(individuals_array):
            chosen_features = [index for index in range(X_train.shape[1]) if individual[index]==1]
            X_train_copy = X_train.iloc[:,chosen_features]
            X_valid_copy = X_valid.iloc[:,chosen_features]
            score = self.objective_function(model,X_train_copy,y_train,X_valid_copy,y_valid)
            if score< self.best_score:
              self.best_score=score
              self.best_score_dimension=individual
            scores.append(score)
        return np.array(scores)

  def sigmoid(self,x):
    return 1/(1+np.exp( -1*(x) ))
        
  def fit(self,model,X_train,y_train,X_valid,y_valid,limit=0.2,MAX=5):
    self.initialize_population(X_train)
    self.best_score=np.inf
    self.best_score_dimension=np.ones(X_train.shape[1]) 
    self.max_limit=np.zeros(X_train.shape[1])
    self.limit=limit   
    self.MAX=MAX

    for i in range(self.n_iteration):
      if (self.food_source.sum(axis=1)==0).sum()>0:
        print((self.food_source.sum(axis=1)==0).sum(),' individuals went zero')
        self.food_source[self.food_source.sum(axis=1)==0]=np.random.randint(0,2,(self.food_source[self.food_source.sum(axis=1)==0].shape[0],\
                                                                                X_train.shape[1] ))
      self.evaluate_fitness(model,X_train,y_train,X_valid,y_valid)
      self.neighbour_food_source=np.where(np.random.uniform(size=(X_train.shape[1],X_train.shape[1]))<self.limit,1,self.food_source)
      self.neighbour_fitness_scores=self.return_fitness(model,X_train,y_train,X_valid,y_valid,self.neighbour_food_source)

      better_source_indexes=np.where(self.neighbour_fitness_scores<self.fitness_scores)[0]
      worse_source_indexes=np.where(self.neighbour_fitness_scores>self.fitness_scores)[0]

      self.max_limit[worse_source_indexes]=self.max_limit[worse_source_indexes]+1
      self.max_limit[better_source_indexes]=0
      self.fitness_scores[better_source_indexes]=self.neighbour_fitness_scores[better_source_indexes]
      self.food_source[better_source_indexes]=self.neighbour_food_source[better_source_indexes]

      if (self.food_source.sum(axis=1)==0).sum()>0:
        print((self.food_source.sum(axis=1)==0).sum(),' individuals went zero')
        self.food_source[self.food_source.sum(axis=1)==0]=np.random.randint(0,2,(self.food_source[self.food_source.sum(axis=1)==0].shape[0],\
                                                                                X_train.shape[1] ))

      indx=np.arange(X_train.shape[1])
      prob=(np.array(self.fitness_scores-self.fitness_scores.max())/(self.fitness_scores.min()-self.fitness_scores.max()))/(np.array(self.fitness_scores-self.fitness_scores.max())/(self.fitness_scores.min()-self.fitness_scores.max())).sum()
      onlooker_phase_indexes=np.random.choice(indx,X_train.shape[1],p=prob)

      self.onlooker_phase_indivduals=np.where(np.random.uniform(size=(X_train.shape[1],X_train.shape[1]))<self.limit,1,self.food_source[onlooker_phase_indexes])

      if (self.onlooker_phase_indivduals.sum(axis=1)==0).sum()>0:
        print((self.onlooker_phase_indivduals.sum(axis=1)==0).sum(),' individuals went zero')
        self.onlooker_phase_indivduals[self.onlooker_phase_indivduals.sum(axis=1)==0]=np.random.randint(0,2,(self.onlooker_phase_indivduals[self.onlooker_phase_indivduals.sum(axis=1)==0].shape[0],\
                                                                                X_train.shape[1] ))
      onlooker_phase_fitness=self.return_fitness(model,X_train,y_train,X_valid,y_valid,self.onlooker_phase_indivduals)

      all_unique=np.unique(onlooker_phase_indexes)
      for uni in all_unique:
        locations=np.where(onlooker_phase_indexes==uni)[0]
        self.food_source[uni]=self.onlooker_phase_indivduals[locations[np.argmin(onlooker_phase_fitness[locations])]] 
        self.max_limit[uni]=0
        self.fitness_scores[uni]=np.min(onlooker_phase_fitness[locations])

      max_expire=np.where(self.max_limit<self.MAX)[0]
      self.food_source[max_expire]=np.random.randint(0,2,size=(len(max_expire),X_train.shape[1]))

      print('iteration ',i,' ',np.array(self.fitness_scores).min(),' |mean ',np.array(self.fitness_scores).mean(),' |best_score ',self.best_score)

    return self.best_score_dimension

    


In [76]:
import scipy
from sklearn.metrics import log_loss

def score_model(model,X_train, y_train, X_valid, y_valid):
    model.fit(X_train,y_train)
    # return log_loss(y_valid,model.predict_proba(X_valid))
    P = (model.predict(X_valid) != y_valid).mean()
    alpha=0.01
    # Compute for the objective function
    j = alpha*(X_valid.shape[1]/30)+(1-alpha)*P
    return j
gaf=bee_colony(score_model,50,20)
res=gaf.fit(clf,X_train, y_train,X_test, y_test)

iteration  0   0.03807017543859649  |mean  0.06715204678362573  |best_score  0.03707017543859649
iteration  1   0.04007017543859649  |mean  0.06638947368421053  |best_score  0.03707017543859649
iteration  2   0.03907017543859649  |mean  0.05884093567251461  |best_score  0.03707017543859649
iteration  3   0.021035087719298243  |mean  0.05493333333333333  |best_score  0.020368421052631577
iteration  4   0.03940350877192982  |mean  0.05894093567251462  |best_score  0.020368421052631577
iteration  5   0.03071929824561403  |mean  0.05850701754385964  |best_score  0.020368421052631577
iteration  6   0.02236842105263158  |mean  0.056691228070175445  |best_score  0.020368421052631577
iteration  7   0.041403508771929824  |mean  0.06220292397660819  |best_score  0.020368421052631577
iteration  8   0.039736842105263154  |mean  0.06301520467836258  |best_score  0.020368421052631577
iteration  9   0.04007017543859649  |mean  0.06119005847953215  |best_score  0.020368421052631577
iteration  10   0.0